# Automatic differentiation blog

Let's make some toy languages. They are going to be "lsps" - or not quite lisps. They are missing some key features that most lisps have. They aren't even all Turing complete. Lisp syntax is used, because it's easy to parse, and parsing is not the interest here.

Each language is defined by a value type, an operator type, an atomizer, and an environment.

The environment is a set of predefined operator and values.

The atomizer converts atomic strings parsed from the syntax into atomic values.

Code for a given lsp is parsed into nested, operator-headed lists. Each element of these lists is a list, an operator, or a value (of the correct type for this lsp).

In [177]:
import operator
from functools import reduce
import re
import math

In [511]:
class Lsp(object):
    open_paren = re.compile("\(")
    close_paren = re.compile("\)")
    splitter = re.compile("\s+")

    def __init__(self, value_type, operator_type, atomizer, environment):
        self.value_type = value_type
        self.operator_type = operator_type
        self.atomizer = atomizer
        self.environment = environment
        
    def parse_helper(self, tokens):
        token = tokens.pop(0)
        if token == "(":
            els = []
            while tokens[0] != ")":
                next_expr, tokens = self.parse_helper(tokens)
                els += [next_expr]

            return els, tokens[1:]
        elif token == ")":
            raise ValueError()
        else:
            return self.atomize(token), tokens
        
    def parse(self, tokens):
        parsed, _ = self.parse_helper(tokens)
        return parsed
    
    def atomize(self, token):
        if token in self.environment.keys():
            return self.environment[token]
        return self.atomizer(token)
    
    def tokenize(self, code_str):
        return list(filter(
            lambda x: x is not '', 
            Lsp.splitter.split(Lsp.close_paren.sub(" ) ", Lsp.open_paren.sub(" ( ", code_str)))
        ))
        
    def eval_stack(self, stack):
        if isinstance(stack, self.value_type) or isinstance(stack, self.operator_type):
            # atom
            return stack

        fn = stack[0]

        if isinstance(fn, list):
            # nested function
            return self.eval_stack(self.eval_stack(stack[0]) + stack[1:])

        if isinstance(fn, self.operator_type):
            # eval this function on the args
            return fn(*map(self.eval_stack, stack[1:]))
    
        raise ValueError(f"Unparsable value {fn}")
        
    def eval_str(self, code_str):
        raise NotImplemented()

# lsplsp

Lsplsp is about the simplest we can get. It can run a few functions on Python ints.

In [628]:
class SimpleLsp(Lsp):
    """A DoubleLsp is an Lsp that returns the same value in the forward and backward results."""
    def eval_str(self, code_str):
        stack = self.parse(self.tokenize(code_str))
        evaled = self.eval_stack(stack)
        return (evaled)

In [629]:
LspLspValue = int
class LspLspOperator(object):
    """An lsplsp operator is just a Python function annotated with a symbol."""
    def __init__(self, symbol, function):
        self.symbol = symbol
        self.function = function
    def __call__(self, *args):
        return self.function(*args)
    

In [630]:
int_token = re.compile("^\d+$")


def lsplsp_atomizer(atom):
    """Lsplsp attempts to evaluate anything it can't find in it's environment as a Python int."""
    if int_token.match(atom):
        return int(atom)
    raise ValueError()

In [631]:
lsplsp_sum = LspLspOperator('+', lambda *args: sum(args))
lsplsp_diff = LspLspOperator('-', lambda *args: args[0] - sum(args[1:]))

In [632]:
lsplsp = SimpleLsp(
    LspLspValue,
    LspLspOperator,
    lsplsp_atomizer,
    { f.symbol: f for f in [lsplsp_sum, lsplsp_diff] }
)

In [633]:
lsplsp.eval_str("(+ 5 2 (- 6 3))")

10

## Forward-backward lsps

The rest of the lsps we're going to take a look at are "forward-backward lsps," or fb-lsps for short.

There are two interesting characteristics of fb-lsps: 

1. Evaluation of an fb-lsp expression returns all the intermediate results in a nested structure.
1. Evaluation of an fb-lsp expression results in two results: a 'forward' results and a 'backward' result.

We can reimplement lsplsp as fblsplsp, an fb-lsp that returns the same value in F and B results. 
It's cheating a little to call this an fb-lsp, but we'll do it anyway.

In [634]:
class FBCheatLsp(Lsp):
    """An Lsp that that returns the same value in the forward and backward results."""
    def eval_str(self, code_str):
        stack = self.parse(self.tokenize(code_str))
        evaled = self.eval_stack(stack)
        return (
            evaled,
            evaled
        )

In [635]:
FBLspLspValue = int
class FBLspLspOperator(object):
    """An fblsplsp function returns the argument values in a tuple with the result."""
    def __init__(self, symbol, function):
        self.symbol = symbol
        self.function = function
    def __call__(self, *args):
        return (self.function(*(arg if isinstance(arg, FBLspLspValue) else arg[0] for arg in args)), args)

In [636]:
fblsplsp_sum = FBLspLspOperator('+', lambda *args: sum(args))
fblsplsp_diff = FBLspLspOperator('-', lambda *args: args[0] - sum(args[1:]))

In [637]:
fblsplsp = FBCheatLsp(
    FBLspLspValue,
    FBLspLspOperator,
    lsplsp_atomizer,
    { f.symbol: f for f in [fblsplsp_sum, fblsplsp_diff] }
)

In [639]:
fblsplsp.eval_str("(+ 5 2 (- 6 3))")

((10, (5, 2, (3, (6, 3)))), (10, (5, 2, (3, (6, 3)))))

This isn't very useful or interesting, though it is potentially nice to have those intermediate values on hand. Let's move on.

# gradlsp

Gradlsp is a lsp that returns a graph of convential computation (just like fblsplsp) in the forward result, and returns a graph of gradients taken with respect to the final result in the backward result.

How do we do this? The operator type for gradlsp includes a two functions: one for the output value of the operator at the  arguments and one for the gradient of the operator at these arguments. At each operator application, we store both the value and the gradient in the resulting value.

Any operation that returns a value records, in that returned value, 
the necessary information to compute a gradient tree with respect either that value,
or to some future value generated from it. 
In practice, this forms a graph, and gradient computation requires a topological sort.
However, we require that the compute graph be a DAG, and simply treat it as a tree.
This is very inefficient: if the DAG isn't treelike, we are performing wasted computations.

When computing the backward result, we propogate the gradient of the final value $y$ backwards. This consists of vector multiplication between the original value and the intermediate gradients. At each intermediate value $v$, we obtain an `adjoint` value $\frac{dy}{dv}$, the derivative of $v$ with respect to the final result $y$. 

The detach operation `($ value)` detaches the compute graph of `value` from it, resulting in a new compute graph consisting of `value` only.
The (singular) node of the gradient DAG rooted at `value`, in any case, will be zero.
This value, though it may depend on other values in actuality, is "held constant" from the standpoint of the gradient computation.

Any operation on a value in gradlsp is pure with respect to the boxed value - it does not modify the original value. Furthermore, all operations are pure with respect to the value's gradient graph. Even the detach operation simply returns a new value with a degenerate graph. This ensures that the gradient graph is a DAG: a cycle in the graph would require a changed (impure) value.

Note that if a node's value is "held constant" in some other way - such as through multiplication by zero - this will not detach the gradient tree and create a degenerate tree. However, the gradient of all child nodes with respect to this node may be zero.

Gradlsp is definitely a toy language. 
For instance, implementing an $n$ layer fully connected neural network with $a$ units in the widest layer in gradlsp will require $O(a^n)$ time.

In [561]:
class FBHookLsp(Lsp):
    """An Lsp that computes the forward and backward results using a hook from stack evaluation."""
    def eval_str(self, code_str):
        stack = self.parse(self.tokenize(code_str))
        evaled = self.eval_stack(stack)
        return (
            evaled.forward(),
            evaled.backward()
        )

In [469]:
class GradLspValue(object):
    def __init__(self, value, gradient = [], children = [], symbol = None):
        self.gradient = gradient
        self.children = children
        self.value = value
        self.symbol = symbol
        
    def __repr__(self):
        return f"<GLV {self.value}, gradient={self.gradient}, with {len(self.children)} children>"
        
    def detach(self):
        return GradLspValue(self.value, [], [], self.symbol)
    
    def forward(self):
        return (self.value,) + ((
            [child.forward() for child in self.children],
        ) if len(self.children) > 0 else tuple())
    
    def backward(self, adjoint=1):
        if len(self.children) == 0:
            return adjoint
        return (adjoint,) + ((
            [child.backward(adjoint * derivative) 
             for child, derivative in zip(self.children, self.gradient)],
        ) if len(self.children) > 0 else tuple())

In [470]:
class GradLspOperator(object):
    def __init__(self, symbol):
        self.symbol = symbol
        
    def __repr__(self):
        return f"<GLF {self.symbol}>"
        
    def __call__(self, *args):
        raise NotImplemented()

In [471]:
int_token = re.compile("^\d+$")
float_token = re.compile("^\d+(\.\d*)$")
#string_token = re.compile("^\"([^\"]*)\"")

def gradlsp_atomizer(token):
    if int_token.match(token):
        return GradLspValue(int(token))
    if float_token.match(token):
        return  GradLspValue(float(token))
    raise ValueError(token)

In [472]:
class GradLspOperatorOperator(GradLspOperator):
    def __init__(self, symbol, function, grad_function):
        super().__init__(symbol)
        self.function = function
        self.grad_function = grad_function
        
    def __call__(self, *args):
        result = GradLspValue(
            self.function(*map(lambda glv: glv.value, args)),
            self.grad_function(*map(lambda glv: glv.value, args)),
            args
        )
        
        return result

In [473]:
plus = GradLspOperatorOperator(
    '+',
    lambda *args: sum(args),
    lambda *args: [1,] * len(args)
)

In [474]:
minus = GradLspOperatorOperator(
    '-',
    lambda *args: args[0] - sum(args[1:]),
    lambda *args: [1,] + [-1,] * (len(args) - 1)
)

In [475]:
prod = GradLspOperatorOperator(
    '*',
    lambda *args: reduce(operator.mul, args, 1),
    lambda *args: [reduce(operator.mul, (args[:i] + args[i+1:]), 1) for i in range(len(args))]
)

In [476]:
quot = GradLspOperatorOperator(
    '/',
    lambda x, y: x / y,
    lambda x, y: [1/y, -(x/(math.pow(y, 2)))]
)

In [477]:
sin = GradLspOperatorOperator(
    'sin',
    lambda x: math.sin(x),
    lambda x: [math.cos(x)]
)

In [478]:
cos = GradLspOperatorOperator(
    'cos',
    lambda x: math.cos(x),
    lambda x: [-math.sin(x)]
)

In [479]:
class DetachOperator(GradLspOperator):
    def __call__(self, *args):
        if len(args) == 1:
            return GradLspValue(args[0].value)
        return [GradLspValue(arg.value) for arg in args]

In [480]:
detach = DetachOperator('$')

In [481]:
pi = GradLspValue(math.pi, symbol='pi')

In [482]:
gradlsp = FBHookLsp(
    GradLspValue, 
    GradLspOperator, 
    gradlsp_atomizer, 
    { f.symbol: f for f in [plus, minus, prod, quot, sin, cos, pi, detach] }
)

In [483]:
gradlsp.eval_str("(* (* 3 2) (cos pi))")

((-6.0, [(6, [(3,), (2,)]), (-1.0, [(3.141592653589793,)])]),
 (1, [(-1.0, [-2.0, -3.0]), (6, [-7.347880794884119e-16])]))

## jacoblsp

Gradlsp isn't very useful. 
It works on scalar valued functions only. 
It's hard to implement more than a best fit line with it.

Jacoblsp is our next step.
An operator in jacoblsp maps $\mathbb R^n \to \mathbb R^m$.
Correspondingly, instead of the gradient, jacoblsp computes the Jacobian matrix for each operation.
In the backwards result, we obtain an adjoint tree of the forwards computation.

Each value in jacoblsp is a 1-dimensional array.
An array is expressed as a literal `1,2,3`. Trailing commas are allowed, but spaces are not.
For instance, `(dot 1,2,3 4,5,6)`  will compute the dot product of $[1,2,3]$ with $[4,5,6]$
Note that every value in jacoblsp is an array: scalar values are simply arrays of size one.

Jacoblsp can express arbitrary functions over $R^n$ now, but in practice, doing so can be quite unwieldy. In particular, because we work with 1-dimensional arrays everywhere, we have to somehow keep track of size transformations if we want to compute operations like matrix multiplication. Our next language will address this problem, and is actually very simple to implement on top of Jacoblsp.

Note that Jacoblsp is still very inefficient. We compute every subtree of the compute DAG for every backwards operation. If our DAG isn't treelike, we are performing wasted computations. It's easier to express combinatorically complex programs in jacoblsp than it was in gradlsp, so this problem is more worrying. We'll come back to it soon.

In [328]:
import numpy as np

In [484]:
class JacobLspValue(object):
    def __init__(self, value, jacobians = None, children=[], symbol = None):
        self.jacobians = jacobians
        self.children = children
        self.value = value
        self.symbol = symbol
        
    def __repr__(self):
        return f"<JLV {self.value}, jacobians={self.jacobians}, with {len(self.children)} children>"
    
    def forward(self):
        if not self.children:
            return (self.value,)
        return (self.value, [child.forward() for child in self.children])
    
    def backward(self, adjoint=None):
        if not self.children:
            return (adjoint,)
        if adjoint is None:
            adjoint = np.ones(self.jacobians[0].shape[0])
            
        # adjoint = (1 x m) where m is the size of this vector
        # self.jacobians = (m x len_i) x len(children) where total size is m x n
        return (adjoint, [np.dot(adjoint, child_jacobian) for child_jacobian, child in zip(self.jacobians, self.children)])
               

In [485]:
class JacobLspOperator(object):
    def __init__(self, symbol, function, jacob_function, max_args=None):
        self.function = function
        self.jacob_function = jacob_function
        self.symbol = symbol
        self.max_args = max_args
        
    def __repr__(self):
        return f"<JLF {self.symbol}>"
        
    def __call__(self, *args):
        if self.max_args is not None and len(args) > self.max_args:
            raise ArgumentException()
            
        result = JacobLspValue(
            self.function(*map(lambda jlv: jlv.value, args)),
            self.jacob_function(*map(lambda jlv: jlv.value, args)),
            args
        )
        
        return result

In [486]:
int_arr_token= re.compile("^(\d+,)*\d,?$")
float_arr_token= re.compile("^(\d+(\.\d*)?,)*\d+(\.\d*)?,?$")

def jacoblsp_atomizer(token):
    if int_arr_token.match(token):
        return JacobLspValue(np.array(list(map(int, filter(bool, token.split(',')))), dtype='int'))
    if int_arr_token.match(token):
        return JacobLspValue(np.array(list(map(float, filter(bool, token.split(',')))), dtype='float'))
    raise ValueError(token)

In [487]:
jplus = JacobLspOperator(
    '+',
    lambda *args: np.stack(args).sum(axis=0),
    lambda *args: [np.ones_like(a) for a in args]
)

In [488]:
jminus = JacobLspOperator(
    '-',
    lambda *args: args[0] - np.stack(args[1:]).sum(axis=0),
    lambda *args: [np.ones_like(args[0])] + [np.full_like(a, -1) for a in args[1:]]
)

In [489]:
jprod = JacobLspOperator(
    '*',
    lambda *args: np.stack(args).prod(axis=0),
    lambda *args: [np.stack(args[:i] + args[i+1:]) for i in range(len(args))]
)

In [490]:
jdot = JacobLspOperator(
    '*.',
    lambda x, y: np.dot(x, y),
    lambda x, y: [y.T, x.T],
    max_args = 2
)

In [491]:
jquot = JacobLspOperator(
    '/',
    lambda x, y: x / y,
    lambda *args: [1/y, -(x/(math.pow(y, 2)))],
    max_args = 2
)

In [492]:
jcos = JacobLspOperator(
    'cos',
    lambda x: np.cos(x),
    lambda x: [-np.sin(x)],
    max_args = 1
)

In [493]:
jsin = JacobLspOperator(
    'sin',
    lambda x: np.sin(x),
    lambda x: [np.cos(x)],
    max_args = 1
)

In [496]:
jacoblsp = FBHookLsp(
    JacobLspValue,
    JacobLspOperator,
    jacoblsp_atomizer,
    { f.symbol: f for f in [jplus, jminus, jprod, jquot, jsin, jcos, jdot] }
)

In [497]:
jacoblsp.eval_str('(*. 1,2,3 3,5,6)')

((31, [(array([1, 2, 3]),), (array([3, 5, 6]),)]),
 (array([ 1.,  1.,  1.]), [14.0, 6.0]))